In [8]:
###############################################################################################################################
import numpy as np
import cv2
import os
from PIL import Image
import pickle    
import datetime
from pymongo import MongoClient
import tkinter
  
    #--------------------------------------------------------#
def create_dataset(regno,name,branch,course,college,parents,students):    
    import numpy as np
    import cv2
    import os


    face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')

    BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
    image_dir = os.path.join(BASE_DIR, "images")
    #--------------------------------------------------------#
    def generate_dataset(img,img_id):

        if not os.path.exists(newpath):
            os.makedirs(newpath)

        cv2.imwrite("{}/user.".format(newpath)+str(img_id)+".jpg",img)
        print(newpath)


    #------------------------------------------------------#
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):
        gray_img=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray_img, scaleFactor,minNeighbors)
        coords = []
        for(x,y,w,h) in features:
            cv2.rectangle(img, (x,y), (x+w,y+h), color, 2)
            coords=[x,y,w,h]
        return coords
    #-------------------------------------------------------#

    def detect(img, faceCascade,img_id):
        color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0),"white":(255,255,255)}
        coords = draw_boundary(img, faceCascade,1.1,10,color['blue'], "Face")

        if len(coords) == 4:
            roi_img = img[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]]

            generate_dataset(roi_img,img_id)

        return img


    #----------------------------------------------------#
    dir_name=regno
    newpath = os.path.join(image_dir, dir_name)

    from pymongo import MongoClient
    myclient = MongoClient('localhost',27017)

    mydb = myclient["students"]  #db name
    mycoll = mydb["attendance"]
    
    list={"Regno":regno,"Name":name,"Branch":branch,"Course":course,"College":college,"Parents Phoneno":parents,"students Phoneno":students}
    mycoll.insert_one(list)



    #-----------------------------------------------------#
    video_capture = cv2.VideoCapture(0)

    img_id = 0

    while True:
        _, img = video_capture.read()

        img = detect(img ,face_cascade, img_id)
        cv2.imshow("face detection", img)
        img_id+=1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()


   
    
###############################################################################################################################    
def train_dataset():
    
    
    BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
    image_dir = os.path.join(BASE_DIR, "images")

    face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()

    current_id = 0
    label_ids = {}
    y_labels = []
    x_train = []

    for root, dirs, files in os.walk(image_dir):
        for file in files:
            if file.endswith("png") or file.endswith("jpg"):
                path = os.path.join(root, file)
                label = os.path.basename(os.path.dirname(path)).replace(" ","-").lower()
                

                if not label in label_ids:
                    label_ids[label] = current_id
                    current_id += 1

                id_ = label_ids[label]
                
                pil_image = Image.open(path).convert("L")  #L turns into gray scale
            
                image_array = np.array(pil_image, "uint8")
                

                faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)

                for (x,y,w,h) in faces:
                    roi = image_array[y:y+h, x:x+w]
                    x_train.append(roi)
                    y_labels.append(id_)

    with open("labels.pickle", 'wb') as f:
        pickle.dump(label_ids, f)

    recognizer.train(x_train,np.array(y_labels))
    recognizer.save("trainner.yml")
    
##########################################################################################################################

def mark_attendance(regno):
    
    myclient = MongoClient('localhost',27017)
    
    mydb = myclient["students"]  #db name
    mycoll = mydb["attendance"]
    
    list=datetime.date.today()
    myquery = {"Regno":regno}
    newvalues = {"$set":{"Attendance.{}".format(list):1}}
    mycoll.update_one(myquery,newvalues)
    

###########################################################################################################################
def make_my_attendance():
    face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("trainner.yml")

    labels = {}
    with open("labels.pickle", 'rb') as f:
        og_labels = pickle.load(f)
        labels = {v:k for k,v in og_labels.items()}

    cap = cv2.VideoCapture(0)

    while(True):
        #capture frame by frame
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
        for (x, y, w, h) in faces:
            #print(x,y,w,h)
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = frame[y:y+h, x:x+w]

            #recognize? deep learned model predict keras tensorflow, pytorch scikit learn
            id_, conf = recognizer.predict(roi_gray)
            if conf >= 45 and conf <= 85:
                font = cv2.FONT_HERSHEY_SIMPLEX
                name = labels[id_]
                print(name)
                #add attendance of today
                mark_attendance(name)
                
                color = (255, 255, 255)
                stroke = 2
                cv2.putText(frame, name, (x,y), font,1,color, stroke,cv2.LINE_AA)

            img_item = "my-image.png"
            cv2.imwrite(img_item,roi_gray)
            color = (255, 0 ,0)
            stroke = 2
            end_cord_x = x+w
            end_cord_y = y+h
            cv2.rectangle(frame, (x,y), (end_cord_x, end_cord_y),color,stroke)
        #Display the resulting frame
        cv2.imshow('frame', frame)
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
    #when everything done, release the capture

    cap.release()
    cv2.destroyAllWindows()

#create_dataset("2016ugcs045","sanay","cse","btech","NIT","8295952408","8676002019") 
#train_dataset()
#make_my_attendance()
################################################################################################################################

def add_to_db():
    class Database:
        def __init__(self, master, *args, **kwargs):
            
            self.master = master
            self.heading = Label(master, text="Add to database", font=('arial 40 bold'), fg='steelblue')
            self.heading.place(x=400, y=0)

                    #label and entries for window
            self.reg_l = Label(master, text="Enter Registration No", font=('arial 18 bold'))
            self.reg_l.place(x=0, y=70)

            self.name_l = Label(master, text="Enter Your Name", font=('arial 18 bold'))
            self.name_l.place(x=0, y=120)

            self.branch_l = Label(master, text="Enter Your Branch", font=('arial 18 bold'))
            self.branch_l.place(x=0, y=170)

            self.course_l = Label(master, text="Enter your Course", font=('arial 18 bold'))
            self.course_l.place(x=0, y=220)

            self.college_l = Label(master, text="Enter Your College Name", font=('arial 18 bold'))
            self.college_l.place(x=0, y=270)

            self.parents_phoneno_l = Label(master, text="Enter Parents Phone Number", font=('arial 18 bold'))
            self.parents_phoneno_l.place(x=0, y=320)

            self.your_phoneno_l = Label(master, text="Enter YOUR Phone Number", font=('arial 18 bold'))
            self.your_phoneno_l.place(x=0, y=370)


            self.reg_e = Entry(master, width=25, font=('arial 18 bold'))
            self.reg_e.place(x=380,y=70)

            self.name_e = Entry(master, width=25, font=('arial 18 bold'))
            self.name_e.place(x=380,y=120)

            self.branch_e = Entry(master, width=25, font=('arial 18 bold'))
            self.branch_e.place(x=380,y=170)

            self.course_e = Entry(master, width=25, font=('arial 18 bold'))
            self.course_e.place(x=380,y=220)

            self.college_e = Entry(master, width=25, font=('arial 18 bold'))
            self.college_e.place(x=380,y=270)

            self.parents_phoneno_e = Entry(master, width=25, font=('arial 18 bold'))
            self.parents_phoneno_e.place(x=380,y=320)

            self.your_phoneno_e = Entry(master, width=25, font=('arial 18 bold'))
            self.your_phoneno_e.place(x=380,y=370)
            
            self.btn_add = Button(master, text="Add to Database", width=25, height=2, bg='steelblue', fg='white',command=self.get_items)
            self.btn_add.place(x=520, y=470)
            
            self.btn_clear = Button(master, text="Clear All Fields", width=25, height=2, bg='lightgreen', fg='white',command=self.clear_all)
            self.btn_clear.place(x=350, y=470)
            
            
            self.btn_train = Button(master, text="Train your model", width=25, height=2, bg='steelblue', fg='white',command=train_dataset)
            self.btn_train.place(x=520, y=520)
            
            self.btn_attendance = Button(master, text="Make The Attendance", width=25, height=2, bg='lightgreen', fg='white',command=make_my_attendance)
            self.btn_attendance.place(x=350, y=520)
            
            
            self.tBox = Text(master, width=60, height=18)
            self.tBox.place(x=750, y=70)
            self.tBox.insert(END,"ID has reached upto:? ")
            
        def get_items(self, *ars, **kwargs):
            self.regno = self.reg_e.get()
            self.name = self.name_e.get()
            self.branch = self.branch_e.get()
            self.course = self.course_e.get()
            self.college = self.college_e.get()
            self.parents_phoneno = self.parents_phoneno_e.get()
            self.your_phoneno =self.your_phoneno_e.get()

            create_dataset(self.regno,self.name,self.branch,self.course,self.college,self.parents_phoneno,self.your_phoneno)
            
        def clear_all(self, *args, **kwargs):
            #num = id + 1
            self.reg_e.delete(0, END)
            self.name_e.delete(0, END)
            self.branch_e.delete(0, END)
            self.course_e.delete(0, END)
            self.college_e.delete(0, END)
            self.parents_phoneno_e.delete(0, END)
            self.your_phoneno_e.delete(0, END)
            
        
    root = Tk()
    b = Database(root)
    
    root.geometry("1366x768+0+0")
    root.title("Add to database")
    root.mainloop()
add_to_db()
#train_dataset()

C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RE

C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RE

C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs031
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RE

C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RE

C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RECGNIZATION\Attendance system using face recognition\images\2016ugcs039
C:\Users\asus\Desktop\pcon-ml\New_folder\FACE_RE

2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs031
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs031
2016ugcs031
2016ugcs031
2016ugcs031
2016ugcs031
2016ugcs031
2016ugcs031
2016ugcs031
2016ugcs039
2016ugcs031
2016ugcs039
2016ugcs031
2016ugcs031
2016ugcs039
2016ugcs031
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs031
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs031
2016ugcs039
2016ugcs039
2016ugcs039
2016ugcs031
2016ugcs031
2016ugcs039
2016ugcs031
2016